<a href="https://colab.research.google.com/github/ritesh-chafer/coronavirus-analysis/blob/master/Web_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from datetime import date
today = date.today()
d = today.strftime("%m-%d-%y")
print("date =", d)

date = 06-29-20


In [5]:
cnn_url = "https://edition.cnn.com/world/live-news/coronavirus-pandemic-{}-intl/index.html".format(d)

In [6]:
print(cnn_url)

https://edition.cnn.com/world/live-news/coronavirus-pandemic-06-29-20-intl/index.html


In [7]:
from bs4 import BeautifulSoup
import requests

In [8]:
html = requests.get(cnn_url).text

In [9]:
soup = BeautifulSoup(html)
print(soup.title)

<title data-rh="true">Coronavirus pandemic: Updates from around the world</title>


In [10]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [13]:
for link in soup.find_all("h2"):
  print("headline : {}".format(link.text))
  for ent in nlp(link.text).ents:
    print("\text : {}, Entity :{}".format(ent.text,ent.label_))

headline : What you need to know
headline : "We did open too quickly," California county public health official says
	ext : California county, Entity :GPE
headline : Why has the guidance on wearing face masks changed so much?
headline : Amazon workers go on strike in Germany over coronavirus cases
	ext : Amazon, Entity :ORG
	ext : Germany, Entity :GPE
headline : Florida reports more than 5,000 new coronavirus cases
	ext : Florida, Entity :GPE
	ext : more than 5,000, Entity :CARDINAL
headline : College deans explain how their admissions priorities have changed in the Covid-19 pandemic
headline : New York City hopes to avert labor layoffs in October as a result of budget cuts
	ext : New York City, Entity :GPE
	ext : October, Entity :DATE
headline : New York City's daily Covid-19 indicators continue to be under maximum thresholds, mayor says
	ext : New York City's, Entity :GPE
	ext : daily, Entity :DATE
headline : Indoor dining in NYC "now in question" for next phase of reopening, mayor s

In [14]:
nbc_url = 'https://www.nbcnews.com/health/coronavirus'
cnbc_rss_url = 'https://www.cnbc.com/id/10000108/device/rss/rss.html'

In [20]:
urls = [cnn_url, nbc_url, cnbc_rss_url]
formats = ['html.parser','html.parser','xml']
tags = ['h2', 'h2', 'description']
website = ['CNN', 'NBC','CNBC']

In [21]:
crawl_len=0
for url in urls:
  print("crawling web page ..........{}".format(url))
  response = requests.get(url)
  soup = BeautifulSoup(response.content, formats[crawl_len])
  for link in soup.find_all(tags[crawl_len]):
    if(len(link.text.split(" ")) > 4):
      print("Headline : {}".format(link.text))
      entities = []
      for ent in nlp(link.text).ents:
        print("\tText: {}, Entity:{}".format(ent.text, ent.label_))
  crawl_len = crawl_len+1


crawling web page ..........https://edition.cnn.com/world/live-news/coronavirus-pandemic-06-29-20-intl/index.html
Headline : What you need to know
Headline : 8 people died from coronavirus in New York yesterday 
	Text: 8, Entity:CARDINAL
	Text: New York, Entity:GPE
	Text: yesterday, Entity:DATE
Headline : Broadway shows suspended for rest of year
	Text: Broadway, Entity:FAC
Headline : "We did open too quickly," California county public health official says
	Text: California county, Entity:GPE
Headline : Why has the guidance on wearing face masks changed so much?
Headline : Amazon workers go on strike in Germany over coronavirus cases
	Text: Amazon, Entity:ORG
	Text: Germany, Entity:GPE
Headline : Florida reports more than 5,000 new coronavirus cases
	Text: Florida, Entity:GPE
	Text: more than 5,000, Entity:CARDINAL
Headline : College deans explain how their admissions priorities have changed in the Covid-19 pandemic
Headline : New York City hopes to avert labor layoffs in October as a 

In [27]:
crawl_len = 0
news_dict = []
for url in urls:
  response = requests.get(url)
  soup = BeautifulSoup(response.content, formats[crawl_len])
  for link in soup.find_all(tags[crawl_len]):
    if(len(link.text.split(" ")) > 4):
      entities = []
      entities = [(ent.text, ent.label_) for ent in nlp(link.text).ents]
      news_dict.append({'website':website[crawl_len],'url':url,'headline':link.text,'entities':entities})
  crawl_len = crawl_len + 1    

In [28]:
news_dict

[{'entities': [],
  'headline': 'What you need to know',
  'url': 'https://edition.cnn.com/world/live-news/coronavirus-pandemic-06-29-20-intl/index.html',
  'website': 'CNN'},
 {'entities': [('Coronavirus', 'ORG')],
  'headline': 'Coronavirus pandemic "not even close to being over," WHO says ',
  'url': 'https://edition.cnn.com/world/live-news/coronavirus-pandemic-06-29-20-intl/index.html',
  'website': 'CNN'},
 {'entities': [('VMAs', 'ORG'),
   ('Brooklyn', 'GPE'),
   ('August', 'DATE'),
   ('New York', 'GPE')],
  'headline': 'VMAs will be held in Brooklyn in August, New York governor says',
  'url': 'https://edition.cnn.com/world/live-news/coronavirus-pandemic-06-29-20-intl/index.html',
  'website': 'CNN'},
 {'entities': [('New York', 'GPE'),
   ('Wednesday', 'DATE'),
   ('NYC', 'ORG'),
   ('3', 'CARDINAL'),
   ('Cuomo', 'ORG')],
  'headline': 'New York will decide by Wednesday whether NYC can have indoor dining in phase 3, Cuomo says',
  'url': 'https://edition.cnn.com/world/live-ne

In [30]:
import pandas as pd


In [31]:
news_df = pd.DataFrame(news_dict)

In [33]:
pd.set_option('max_colwidth', 800)
news_df.head(30)

,website,url,headline,entities
0,CNN,https://edition.cnn.com/world/live-news/coronavirus-pandemic-06-29-20-intl/index.html,What you need to know,[]
1,CNN,https://edition.cnn.com/world/live-news/coronavirus-pandemic-06-29-20-intl/index.html,"Coronavirus pandemic ""not even close to being over,"" WHO says","[(Coronavirus, ORG)]"
2,CNN,https://edition.cnn.com/world/live-news/coronavirus-pandemic-06-29-20-intl/index.html,"VMAs will be held in Brooklyn in August, New York governor says","[(VMAs, ORG), (Brooklyn, GPE), (August, DATE), (New York, GPE)]"
3,CNN,https://edition.cnn.com/world/live-news/coronavirus-pandemic-06-29-20-intl/index.html,"New York will decide by Wednesday whether NYC can have indoor dining in phase 3, Cuomo says","[(New York, GPE), (Wednesday, DATE), (NYC, ORG), (3, CARDINAL), (Cuomo, ORG)]"
4,CNN,https://edition.cnn.com/world/live-news/coronavirus-pandemic-06-29-20-intl/index.html,8 people died from coronavirus in New York yesterday,"[(8, CARDINAL), (New York, GPE), (yesterday, DATE)]"
5,CNN,https://edition.cnn.com/world/live-news/coronavirus-pandemic-06-29-20-intl/index.html,Broadway shows suspended for rest of year,"[(Broadway, FAC)]"
6,CNN,https://edition.cnn.com/world/live-news/coronavirus-pandemic-06-29-20-intl/index.html,"""We did open too quickly,"" California county public health official says","[(California county, GPE)]"
7,CNN,https://edition.cnn.com/world/live-news/coronavirus-pandemic-06-29-20-intl/index.html,Why has the guidance on wearing face masks changed so much?,[]
8,CNN,https://edition.cnn.com/world/live-news/coronavirus-pandemic-06-29-20-intl/index.html,Amazon workers go on strike in Germany over coronavirus cases,"[(Amazon, ORG), (Germany, GPE)]"
9,CNN,https://edition.cnn.com/world/live-news/coronavirus-pandemic-06-29-20-intl/index.html,"Florida reports more than 5,000 new coronavirus cases","[(Florida, GPE), (more than 5,000, CARDINAL)]"


In [34]:
pd.concat([news_df[['website','url','headline']],news_df['entities'].apply(pd.Series)],axis=1)

,website,url,headline,0,1,2,3,4,5,6
0,CNN,https://edition.cnn.com/world/live-news/coronavirus-pandemic-06-29-20-intl/index.html,What you need to know,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CNN,https://edition.cnn.com/world/live-news/coronavirus-pandemic-06-29-20-intl/index.html,"Coronavirus pandemic ""not even close to being over,"" WHO says","(Coronavirus, ORG)",NaN,NaN,NaN,NaN,NaN,NaN
2,CNN,https://edition.cnn.com/world/live-news/coronavirus-pandemic-06-29-20-intl/index.html,"VMAs will be held in Brooklyn in August, New York governor says","(VMAs, ORG)","(Brooklyn, GPE)","(August, DATE)","(New York, GPE)",NaN,NaN,NaN
3,CNN,https://edition.cnn.com/world/live-news/coronavirus-pandemic-06-29-20-intl/index.html,"New York will decide by Wednesday whether NYC can have indoor dining in phase 3, Cuomo says","(New York, GPE)","(Wednesday, DATE)","(NYC, ORG)","(3, CARDINAL)","(Cuomo, ORG)",NaN,NaN
4,CNN,https://edition.cnn.com/world/live-news/coronavirus-pandemic-06-29-20-intl/index.html,8 people died from coronavirus in New York yesterday,"(8, CARDINAL)","(New York, GPE)","(yesterday, DATE)",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
87,CNBC,https://www.cnbc.com/id/10000108/device/rss/rss.html,Delta and the pilots' labor union reached an agreement for early retirements as the carrier scrambles to reduce headcount.,"(Delta, ORG)",NaN,NaN,NaN,NaN,NaN,NaN
88,CNBC,https://www.cnbc.com/id/10000108/device/rss/rss.html,"""We're always busy in the summertime and what we're seeing now is a typical summer for us,"" Memorial Hermann Health System CEO Dr. David Callender told CNBC.","(Memorial Hermann Health System CEO, WORK_OF_ART)","(David Callender, PERSON)","(CNBC, ORG)",NaN,NaN,NaN,NaN
89,CNBC,https://www.cnbc.com/id/10000108/device/rss/rss.html,"San Francisco Mayor London Breed said in a tweet Friday that the city will temporarily delay its planned reopening for Monday, which included nail salons, barbershops, outdoor bars.","(San Francisco, GPE)","(London Breed, PERSON)","(Friday, DATE)","(Monday, DATE)",NaN,NaN,NaN
90,CNBC,https://www.cnbc.com/id/10000108/device/rss/rss.html,"Ro, which previously specialized in men's health, is now generating $250 million in annual revenue after expanding its product line.","($250 million, MONEY)","(annual, DATE)",NaN,NaN,NaN,NaN,NaN
